In [1]:
import os
import json
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import open3d as o3d
import torch
from torch import nn

from plyfile import PlyData, PlyElement
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cdist

from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### 1. pcm2df and df2pcm converter

In [2]:
class PointCloudModel(nn.Module):
    def __init__(self, max_sh_degree=3, device="cpu"):
        super(PointCloudModel, self).__init__()
        self.max_sh_degree = max_sh_degree
        self._xyz = None
        self._features_dc = None
        self._features_rest = None
        self._opacity = None
        self._scaling = None
        self._rotation = None
        self.pretrained_exposures = None
        self.device = device

    def load_ply(self, path, use_train_test_exp=False):
        """
        Additional Explanation of the point cloud model's features:
        |---------------------------------------------------------------------------------------------------------------------------------------------------|
        | Variable              | Meaning                                                 | Typical Shape (if RGB)                                          |
        | --------------------- | ------------------------------------------------------- | --------------------------------------------------------------- |
        | `self._features_dc`   | Zeroth-order SH coefficients (constant per point color) | `[N_points, 3]`                                                 |
        | `self._features_rest` | Higher-order SH coefficients (view-dependent features)  | `[N_points, 3 x (n_SH - 1)]` where `n_SH = (max_sh_degree+1)^2` |
        |---------------------------------------------------------------------------------------------------------------------------------------------------|
        """
        plydata = PlyData.read(path)
        if use_train_test_exp:
            exposure_file = os.path.join(os.path.dirname(path), os.pardir, os.pardir, "exposure.json")
            if os.path.exists(exposure_file):
                with open(exposure_file, "r") as f:
                    exposures = json.load(f)
                self.pretrained_exposures = {image_name: torch.FloatTensor(exposures[image_name]).requires_grad_(False).cuda() for image_name in exposures}
                print(f"Pretrained exposures loaded.")
            else:
                print(f"No exposure to be loaded at {exposure_file}")
                self.pretrained_exposures = None

        xyz = np.stack((np.asarray(plydata.elements[0]["x"]),
                        np.asarray(plydata.elements[0]["y"]),
                        np.asarray(plydata.elements[0]["z"])),  axis=1)
        opacities = np.asarray(plydata.elements[0]["opacity"])[..., np.newaxis]

        features_dc = np.zeros((xyz.shape[0], 3, 1))
        features_dc[:, 0, 0] = np.asarray(plydata.elements[0]["f_dc_0"])
        features_dc[:, 1, 0] = np.asarray(plydata.elements[0]["f_dc_1"])
        features_dc[:, 2, 0] = np.asarray(plydata.elements[0]["f_dc_2"])

        # extra_f_names = [p.name for p in plydata.elements[0].properties if p.name.startswith("f_rest_")]
        # extra_f_names = sorted(extra_f_names, key = lambda x: int(x.split('_')[-1]))
        # assert len(extra_f_names)==3*(self.max_sh_degree + 1) ** 2 - 3
        # features_extra = np.zeros((xyz.shape[0], len(extra_f_names)))
        # for idx, attr_name in enumerate(extra_f_names):
        #     features_extra[:, idx] = np.asarray(plydata.elements[0][attr_name])
        # Reshape (P,F*SH_coeffs) to (P, F, SH_coeffs except DC)
        # features_extra = features_extra.reshape((features_extra.shape[0], 3, (self.max_sh_degree + 1) ** 2 - 1))

        scale_names = [p.name for p in plydata.elements[0].properties if p.name.startswith("scale_")]
        scale_names = sorted(scale_names, key = lambda x: int(x.split('_')[-1]))
        scales = np.zeros((xyz.shape[0], len(scale_names)))
        for idx, attr_name in enumerate(scale_names):
            scales[:, idx] = np.asarray(plydata.elements[0][attr_name])

        rot_names = [p.name for p in plydata.elements[0].properties if p.name.startswith("rot")]
        rot_names = sorted(rot_names, key = lambda x: int(x.split('_')[-1]))
        rots = np.zeros((xyz.shape[0], len(rot_names)))
        for idx, attr_name in enumerate(rot_names):
            rots[:, idx] = np.asarray(plydata.elements[0][attr_name])

        self._xyz = nn.Parameter(torch.tensor(xyz, dtype=torch.float, device=self.device).requires_grad_(True))
        self._features_dc = nn.Parameter(torch.tensor(features_dc, dtype=torch.float, device=self.device).transpose(1, 2).contiguous().requires_grad_(True))
        # self._features_rest = nn.Parameter(torch.tensor(features_extra, dtype=torch.float, device=self.device).transpose(1, 2).contiguous().requires_grad_(True))
        self._opacity = nn.Parameter(torch.tensor(opacities, dtype=torch.float, device=self.device).requires_grad_(True))
        self._scaling = nn.Parameter(torch.tensor(scales, dtype=torch.float, device=self.device).requires_grad_(True))
        self._rotation = nn.Parameter(torch.tensor(rots, dtype=torch.float, device=self.device).requires_grad_(True))

        self.active_sh_degree = self.max_sh_degree


    def convert_ply_to_df(self, beta, alpha, id, df_max_size=50000):
        
        data_dict = {
            'position': self._xyz.detach().numpy(),
            'f_dc': self._features_dc.detach().numpy(),
            # 'f_rest': self._features_rest.detach().numpy(),
            'opacity': self._opacity.detach().numpy(),
            'scale': self._scaling.detach().numpy(),
            'rotation': self._rotation.detach().numpy(),
            'beta': beta * np.ones(self._opacity.detach().numpy().shape),
            'alpha': alpha * np.ones(self._opacity.detach().numpy().shape),
            'is_fog': (beta!=0) * np.ones(self._opacity.detach().numpy().shape),
            'id': id * np.ones(self._opacity.detach().numpy().shape)
        }

        
        # Flatten each into separate columns
        df_dict = {}

        # Flatten position
        for i, axis in enumerate(['x', 'y', 'z']):
            df_dict[f'pos_{axis}'] = data_dict['position'][:, i]

        # Flatten feature_dc
        for i, color in enumerate(['0', '1', '2']):
            df_dict[f'f_dc_{color}'] = data_dict['f_dc'][:, 0, i]

        # Flatten scale
        for i, axis in enumerate(['x', 'y', 'z']):
            df_dict[f'scale_{axis}'] = data_dict['scale'][:, i]

        # Flatten rotation
        for i, axis in enumerate(['0', '1', '2', '3']):
            df_dict[f'rot_{axis}'] = data_dict['rotation'][:, i]

        # Opacity
        df_dict['opacity'] = data_dict['opacity'].squeeze()

        # Flatten feature_rest
        df_dict['beta'] = data_dict['beta'].squeeze()
        df_dict['alpha'] = data_dict['alpha'].squeeze()
        df_dict['is_fog'] = data_dict['is_fog'].squeeze()
        df_dict['id'] = data_dict['id'].squeeze()

        return pd.DataFrame(df_dict)
    

    def convert_df_to_ply(self, df, output_path, normals=None):
        # means, scales, rotations, rgbs, opacities, normals=None):
        means = df[['pos_x', 'pos_y', 'pos_z']].values
        scales = df[['scale_x', 'scale_y', 'scale_z']].values
        rotations = df[['rot_0', 'rot_1', 'rot_2', 'rot_3']].values
        colors = df[['f_dc_0', 'f_dc_1', 'f_dc_2']].values
        opacities = df['opacity'].values[..., np.newaxis]

        if normals is None:
            normals = np.zeros_like(means)

        if scales.shape[1] == 1:
            scales = np.tile(scales, (1, 3))

        attrs = ['x', 'y', 'z',
                'nx', 'ny', 'nz',
                'f_dc_0', 'f_dc_1', 'f_dc_2',
                'opacity',
                'scale_0', 'scale_1', 'scale_2',
                'rot_0', 'rot_1', 'rot_2', 'rot_3',]

        dtype_full = [(attribute, 'f4') for attribute in attrs]
        elements = np.empty(means.shape[0], dtype=dtype_full)

        attributes = np.concatenate((means, normals, colors, opacities, scales, rotations), axis=1)
        elements[:] = list(map(tuple, attributes))
        el = PlyElement.describe(elements, 'vertex')
        PlyData([el]).write(output_path)

        print(f"Saved PLY format Splat to {output_path}")


### 2.1. Shrink the range of PCMs 

#### a. train

REFERENCE:

1. SMALL Scene
- x_range = [-4.0, 3.0]
- y_range = [-4.7, 1.0]
- z_range = [-2.0, 4.0]

2. MIDDLE Scene
- x_range = [-9.0, 9.0]
- y_range = [-6.0, 1.2]
- z_range = [-4.5, 7.0]

In [4]:
PLY_PATHs = glob("/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase1/train/train*.ply")

# Load the point cloud
pcms = {}
for i, PLY_PATH in enumerate(PLY_PATHs):
    print(f"Loading {PLY_PATH}...")
    key = PLY_PATH.split('/train/train_')[-1].split('.')[0]
    pcm = PointCloudModel(max_sh_degree=3)
    pcm.load_ply(PLY_PATH, use_train_test_exp=False)
    pcms[key] = pcm

pcms = dict(
    sorted(
        pcms.items(),
        key=lambda kv: (
            int(kv[0].split('_')[0].replace('beta', '')),
            int(kv[0].split('_')[1].replace('alpha', ''))
        )
    )
)


len(pcms)

Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase1/train/train_beta1_alpha250.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase1/train/train_beta5_alpha100.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase1/train/train_beta3_alpha125.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase1/train/train_beta2_alpha150.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase1/train/train_beta3_alpha225.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase1/train/train_beta3_alpha250.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase1/train/train_beta3_alpha175.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase1/train/train_beta1_alpha025.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase1/train/tr

61

In [5]:
target_dir = "/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/train"

for fog_params, pcm in pcms.items():
    beta_str, alpha_str = fog_params.split('_')
    beta = int(beta_str.replace("beta", ""))
    alpha = int(alpha_str.replace("alpha", ""))

    df_pcm = pcm.convert_ply_to_df(beta=beta, alpha=alpha, id=-1)

    condition_x = (df_pcm['pos_x'] >= -4.0) * (df_pcm['pos_x'] <= 3.0)
    condition_y = (df_pcm['pos_y'] >= -4.7) * (df_pcm['pos_y'] <= 1.0)
    condition_z = (df_pcm['pos_z'] >= -2.0) * (df_pcm['pos_z'] <= 4.0)

    condition_all = condition_x * condition_y * condition_z

    df_filtered_pcm = df_pcm[condition_all]

    pcm.convert_df_to_ply(df_filtered_pcm, output_path=f"{target_dir}/train_{fog_params}.ply")

Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/train/train_beta0_alpha000.ply
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/train/train_beta1_alpha025.ply
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/train/train_beta1_alpha050.ply
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/train/train_beta1_alpha075.ply
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/train/train_beta1_alpha100.ply
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/train/train_beta1_alpha125.ply
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/train/train_beta1_alpha150.ply
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2

#### b. truck

REFERENCE:

- x_range = [-9.0, 9.0]
- y_range = [-9.0, 1.6]
- z_range = [-9.0, 9.0]

In [5]:
# PLY_PATHs = glob("/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase1/truck/truck*.ply")
PLY_PATHs = ["/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase1/truck/truck_beta0_alpha000.ply"]

# Load the point cloud
pcms = {}
for i, PLY_PATH in enumerate(PLY_PATHs):
    print(f"Loading {PLY_PATH}...")
    key = PLY_PATH.split('/truck/truck_')[-1].split('.')[0]
    pcm = PointCloudModel(max_sh_degree=3)
    pcm.load_ply(PLY_PATH, use_train_test_exp=False)
    pcms[key] = pcm

pcms = dict(
    sorted(
        pcms.items(),
        key=lambda kv: (
            int(kv[0].split('_')[0].replace('beta', '')),
            int(kv[0].split('_')[1].replace('alpha', ''))
        )
    )
)


len(pcms)

Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase1/truck/truck_beta0_alpha000.ply...


1

In [6]:
target_dir = "/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/truck"

for fog_params, pcm in pcms.items():
    beta_str, alpha_str = fog_params.split('_')
    beta = int(beta_str.replace("beta", ""))
    alpha = int(alpha_str.replace("alpha", ""))

    df_pcm = pcm.convert_ply_to_df(beta=beta, alpha=alpha, id=-1)

    condition_x = (df_pcm['pos_x'] >= -9.0) * (df_pcm['pos_x'] <= 9.0)
    condition_y = (df_pcm['pos_y'] >= -9.0) * (df_pcm['pos_y'] <= 1.6)
    condition_z = (df_pcm['pos_z'] >= -9.0) * (df_pcm['pos_z'] <= 9.0)

    condition_all = condition_x * condition_y * condition_z

    df_filtered_pcm = df_pcm[condition_all]

    pcm.convert_df_to_ply(df_filtered_pcm, output_path=f"{target_dir}/truck_{fog_params}.ply")

Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/truck/truck_beta0_alpha000.ply


#### c. drjohnson

REFERENCE:

- x_range = [-7.50, 6.20]
- y_range = [-2.70, 3.40]
- z_range = [-2.00, 2.00]

In [ ]:
PLY_PATHs = glob("/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase1/drjohnson/drjohnson*.ply")

# Load the point cloud
pcms = {}
for i, PLY_PATH in enumerate(PLY_PATHs):
    print(f"Loading {PLY_PATH}...")
    key = PLY_PATH.split('/drjohnson/drjohnson_')[-1].split('.')[0]
    pcm = PointCloudModel(max_sh_degree=3)
    pcm.load_ply(PLY_PATH, use_train_test_exp=False)
    pcms[key] = pcm

pcms = dict(
    sorted(
        pcms.items(),
        key=lambda kv: (
            int(kv[0].split('_')[0].replace('beta', '')),
            int(kv[0].split('_')[1].replace('alpha', ''))
        )
    )
)


len(pcms)

Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase1/drjohnson/drjohnson_beta0_alpha000.ply...


1

In [32]:
target_dir = "/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/drjohnson"

for fog_params, pcm in pcms.items():
    beta_str, alpha_str = fog_params.split('_')
    beta = int(beta_str.replace("beta", ""))
    alpha = int(alpha_str.replace("alpha", ""))

    df_pcm = pcm.convert_ply_to_df(beta=beta, alpha=alpha, id=-1)

    condition_x = (df_pcm['pos_x'] >= -7.5) * (df_pcm['pos_x'] <= -2.2)
    condition_y = (df_pcm['pos_y'] >= -2.7) * (df_pcm['pos_y'] <= 3.0)
    condition_z = (df_pcm['pos_z'] >= -2.0) * (df_pcm['pos_z'] <= 2.0)
    condition1 = condition_x * condition_y * condition_z

    condition_x = (df_pcm['pos_x'] >= -2.2) * (df_pcm['pos_x'] <= 0.7)
    condition_y = (df_pcm['pos_y'] >= -1.8) * (df_pcm['pos_y'] <= 1.6)
    condition_z = (df_pcm['pos_z'] >= -2.0) * (df_pcm['pos_z'] <= 2.0)
    condition2 = condition_x * condition_y * condition_z

    condition_x = (df_pcm['pos_x'] >=  0.7) * (df_pcm['pos_x'] <= 6.2)
    condition_y = (df_pcm['pos_y'] >= -2.7) * (df_pcm['pos_y'] <= 3.4)
    condition_z = (df_pcm['pos_z'] >= -2.0) * (df_pcm['pos_z'] <= 2.0)
    condition3 = condition_x * condition_y * condition_z

    df_filtered_pcm = df_pcm[condition1 + condition2 + condition3]

    pcm.convert_df_to_ply(df_filtered_pcm, output_path=f"{target_dir}/drjohnson_{fog_params}.ply")

Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/drjohnson/drjohnson_beta0_alpha000.ply


#### d. playroom

REFERENCE:

- x_range = [-6.60, 5.40]
- y_range = [-4.35, 4.05]
- z_range = [-5.05, 5.45]

In [ ]:
PLY_PATHs = glob("/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase1/playroom/playroom*.ply")

# Load the point cloud
pcms = {}
for i, PLY_PATH in enumerate(PLY_PATHs):
    print(f"Loading {PLY_PATH}...")
    key = PLY_PATH.split('/playroom/playroom_')[-1].split('.')[0]
    pcm = PointCloudModel(max_sh_degree=3)
    pcm.load_ply(PLY_PATH, use_train_test_exp=False)
    pcms[key] = pcm

pcms = dict(
    sorted(
        pcms.items(),
        key=lambda kv: (
            int(kv[0].split('_')[0].replace('beta', '')),
            int(kv[0].split('_')[1].replace('alpha', ''))
        )
    )
)


len(pcms)

Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase1/playroom/playroom_beta0_alpha000.ply...


1

In [34]:
target_dir = "/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/playroom"

for fog_params, pcm in pcms.items():
    beta_str, alpha_str = fog_params.split('_')
    beta = int(beta_str.replace("beta", ""))
    alpha = int(alpha_str.replace("alpha", ""))

    df_pcm = pcm.convert_ply_to_df(beta=beta, alpha=alpha, id=-1)

    condition_x = (df_pcm['pos_x'] >= -6.60) * (df_pcm['pos_x'] <= 5.40)
    condition_y = (df_pcm['pos_y'] >= -4.35) * (df_pcm['pos_y'] <= 4.05)
    condition_z = (df_pcm['pos_z'] >= -5.05) * (df_pcm['pos_z'] <= 5.45)

    condition_all = condition_x * condition_y * condition_z

    df_filtered_pcm = df_pcm[condition_all]

    pcm.convert_df_to_ply(df_filtered_pcm, output_path=f"{target_dir}/playroom_{fog_params}.ply")

Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/playroom/playroom_beta0_alpha000.ply


### 2.2. Labeling fog gaussians and the others

#### Deprecated

##### a. Fog Gaussians + Other Gaussians = All Gaussians

In [59]:
# a-1. Initialize: Fog Gaussians = All Gaussians - Other Gaussians

PLY_PATH_all    = "/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/truck/truck_beta4_alpha025.ply"
PLY_PATH_others = "/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/other_gaussian/truck_beta4_alpha025.ply"

ref_cols = ['pos_x', 'pos_y', 'pos_z']

pcm_all    = PointCloudModel(max_sh_degree=3)
pcm_others = PointCloudModel(max_sh_degree=3)

pcm_all.load_ply(PLY_PATH_all, use_train_test_exp=False)
pcm_others.load_ply(PLY_PATH_others, use_train_test_exp=False)

df_pcm_all = pcm_all.convert_ply_to_df(beta=2.0, alpha=250, id=-1)
df_pcm_others = pcm_others.convert_ply_to_df(beta=2.0, alpha=250, id=-1)

# Drop matching rows
df_pcm_fog = df_pcm_all[~df_pcm_all[ref_cols].apply(tuple, axis=1).isin(df_pcm_others[ref_cols].apply(tuple, axis=1))]

In [60]:
pcm_fog = PointCloudModel(max_sh_degree=3)
PLY_PATH_fog = "/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta4_alpha025.ply"

pcm_fog.convert_df_to_ply(df_pcm_fog, output_path=PLY_PATH_fog)

Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta4_alpha025.ply


In [93]:
# a-2. Refinement: Other Gaussians = All Gaussians - Fog Gaussians

PLY_PATH_all = "/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/train/train_beta2_alpha250.ply"
PLY_PATH_fog = "/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/fog_gaussian/train_beta2_alpha250.ply"

ref_cols = ['pos_x', 'pos_y', 'pos_z']
ref_cols = ['pos_x', 'pos_y', 'pos_z']

pcm_all = PointCloudModel(max_sh_degree=3)
pcm_fog = PointCloudModel(max_sh_degree=3)

pcm_all.load_ply(PLY_PATH_all, use_train_test_exp=False)
pcm_fog.load_ply(PLY_PATH_fog, use_train_test_exp=False)

df_pcm_all = pcm_all.convert_ply_to_df(beta=2.0, alpha=250, id=-1)
df_pcm_fog = pcm_fog.convert_ply_to_df(beta=2.0, alpha=250, id=-1)

# Drop matching rows
df_pcm_others = df_pcm_all[~df_pcm_all[ref_cols].apply(tuple, axis=1).isin(df_pcm_fog[ref_cols].apply(tuple, axis=1))]

In [94]:
pcm_others = PointCloudModel(max_sh_degree=3)
PLY_PATH_others = "/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/other_gaussian/train_beta2_alpha250.ply"

pcm_others.convert_df_to_ply(df_pcm_others, output_path=PLY_PATH_others)

Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/other_gaussian/train_beta2_alpha250.ply


##### b. Apply one to all datasets

In [44]:
# ------------------------------
# Helper functions
# ------------------------------
def quaternion_to_rotation_matrix(q):
    """
    Convert quaternion (w, x, y, z) to a 3x3 rotation matrix.
    """
    w, x, y, z = q
    return np.array([
        [1 - 2*(y**2 + z**2),   2*(x*y - z*w),       2*(x*z + y*w)],
        [2*(x*y + z*w),         1 - 2*(x**2 + z**2), 2*(y*z - x*w)],
        [2*(x*z - y*w),         2*(y*z + x*w),       1 - 2*(x**2 + y**2)]
    ])


def get_aabb_from_transform(pos, quat, scale):
    """
    Given position (3,), quaternion (4,), and scale (3,),
    compute the axis-aligned bounding box after rotation.
    Returns (min_bounds, max_bounds)
    """
    # Half extents in local coordinates
    sx, sy, sz = scale / 2.0

    # 8 corners in local space
    corners_local = np.array([
        [ sx,  sy,  sz],
        [ sx,  sy, -sz],
        [ sx, -sy,  sz],
        [ sx, -sy, -sz],
        [-sx,  sy,  sz],
        [-sx,  sy, -sz],
        [-sx, -sy,  sz],
        [-sx, -sy, -sz]
    ])

    # Rotate
    R = quaternion_to_rotation_matrix(quat)
    corners_rot = corners_local @ R.T

    # Translate to world space
    corners_world = corners_rot + pos

    # Axis-aligned bounding box
    min_bounds = corners_world.min(axis=0)
    max_bounds = corners_world.max(axis=0)

    return min_bounds, max_bounds


def filter_points_in_aabbs(points_df, objects_df):
    """
    Given a point cloud and object transforms, return points inside any object's AABB.
    """
    keep_mask = np.zeros(len(points_df), dtype=bool)

    for _, row in tqdm(objects_df.iterrows(), total=len(objects_df)):
        pos = np.array([row["pos_x"], row["pos_y"], row["pos_z"]])
        quat = np.array([row["rot_0"], row["rot_1"], row["rot_2"], row["rot_3"]])
        scale = np.array([abs(row["scale_x"]), abs(row["scale_y"]), abs(row["scale_z"])])  # ensure positive

        min_bounds, max_bounds = get_aabb_from_transform(pos, quat, scale)
        if np.max(max_bounds - min_bounds) > 10.0:
            pass
        else:
            mask = (
                (points_df["pos_x"] >= min_bounds[0]) & (points_df["pos_x"] <= max_bounds[0]) &
                (points_df["pos_y"] >= min_bounds[1]) & (points_df["pos_y"] <= max_bounds[1]) &
                (points_df["pos_z"] >= min_bounds[2]) & (points_df["pos_z"] <= max_bounds[2])
            )
            keep_mask |= mask  # points inside ANY object

    return points_df[keep_mask]

def filter_points_in_boxes(points_df, objects_df, length=0.2, batch_size=4096, device="cuda"):
    """
    GPU-accelerated filtering of points inside any object's AABB.
    Returns a filtered pandas DataFrame.
    """
    # Convert to torch tensors on GPU
    points = torch.tensor(points_df[["pos_x", "pos_y", "pos_z"]].to_numpy(),
                          dtype=torch.float32, device=device)
    objects = torch.tensor(objects_df[["pos_x", "pos_y", "pos_z"]].to_numpy(),
                           dtype=torch.float32, device=device)

    keep_mask = torch.zeros(points.shape[0], dtype=torch.bool, device=device)

    # Process objects in batches
    for start in tqdm(range(0, objects.shape[0], batch_size)):
        objs_batch = objects[start:start + batch_size]

        mins = objs_batch - length
        maxs = objs_batch + length

        inside = ((points[:, None, :] >= mins[None, :, :]) &
                  (points[:, None, :] <= maxs[None, :, :])).all(dim=2)

        keep_mask |= inside.any(dim=1)

        if keep_mask.all():
            break

    # Move mask back to CPU numpy
    keep_mask_np = keep_mask.cpu().numpy()
    print(f"{sum(keep_mask_np)} / {len(keep_mask_np)}")

    return points_df[keep_mask_np]

In [ ]:
PLY_PATH_all    = "/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/truck/truck_beta4_alpha025.ply"
PLY_PATH_others = "/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/other_gaussian/truck_beta0_alpha000.ply"

pcm_all = PointCloudModel(max_sh_degree=3)
pcm_others = PointCloudModel(max_sh_degree=3)

pcm_all.load_ply(PLY_PATH_all, use_train_test_exp=False)
pcm_others.load_ply(PLY_PATH_others, use_train_test_exp=False)

df_pcm_all = pcm_all.convert_ply_to_df(beta=2.0, alpha=25, id=-1)
df_pcm_others = pcm_others.convert_ply_to_df(beta=2.0, alpha=250, id=-1)

pcm_others_new = PointCloudModel(max_sh_degree=3)
df_pcm_others_new = filter_points_in_boxes(df_pcm_all, df_pcm_others, length=0.2, batch_size=3064)
pcm_others_new.convert_df_to_ply(df_pcm_others_new, output_path=f"/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/other_gaussian/truck_beta4_alpha025.ply")

torch.cuda.empty_cache()

100%|██████████| 285/285 [00:03<00:00, 79.49it/s]


144376 / 147759
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/other_gaussian/truck_beta4_alpha025.ply


In [83]:
df_pcm_fog.head(2)

,pos_x,pos_y,pos_z,f_dc_0,f_dc_1,f_dc_2,scale_x,scale_y,scale_z,rot_0,rot_1,rot_2,rot_3,opacity,beta,alpha,is_fog,id
0,-3.742636,-2.300430,-4.493582,0.852467,1.138692,1.646362,-1.850119,-8.685864,-0.227652,0.318821,0.438886,0.782133,-0.187718,9.994455,2.0,250.0,1.0,-1.0
1,-3.868190,-2.240141,-4.297435,0.882945,1.188643,1.574612,-1.510371,-5.218655,-0.203309,0.371921,0.336907,0.705750,-0.242891,8.655488,2.0,250.0,1.0,-1.0


np.float32(8.923532)

In [64]:

# Paths
BASE_DIR = "/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed"
PHASE2_DIR = os.path.join(BASE_DIR, "phase2/truck")
PHASE3_OTHERS = os.path.join(BASE_DIR, "phase3/truck/other_gaussian")
PHASE3_FOG = os.path.join(BASE_DIR, "phase3/truck/fog_gaussian")

# Ensure output dirs exist
os.makedirs(PHASE3_OTHERS, exist_ok=True)
os.makedirs(PHASE3_FOG, exist_ok=True)

# Reference "others" ply
PLY_PATH_OTHERS = os.path.join(PHASE3_OTHERS, "truck_beta0_alpha000.ply")

# Iterate through all truck_beta*_alpha*.ply in phase2
ply_files = sorted(glob(os.path.join(PHASE2_DIR, "truck_beta*_alpha*.ply")))

for PLY_PATH_ALL in ply_files:
    fname = os.path.basename(PLY_PATH_ALL)  # e.g. "truck_beta4_alpha025.ply"
    print(f"\nProcessing {fname}...")

    # Load models
    pcm_all = PointCloudModel(max_sh_degree=3)
    pcm_others = PointCloudModel(max_sh_degree=3)

    pcm_all.load_ply(PLY_PATH_ALL, use_train_test_exp=False)
    pcm_others.load_ply(PLY_PATH_OTHERS, use_train_test_exp=False)

    df_pcm_all = pcm_all.convert_ply_to_df(beta=-1, alpha=-1, id=-1)
    df_pcm_others = pcm_others.convert_ply_to_df(beta=-1, alpha=-1, id=-1)

    # Filter points in boxes (GPU accelerated)
    pcm_others_new = PointCloudModel(max_sh_degree=3)
    df_pcm_others_new = filter_points_in_boxes(
        df_pcm_all, df_pcm_others, length=0.2, batch_size=2048, device="cuda"
    )

    # Save filtered "others"
    output_others_path = os.path.join(PHASE3_OTHERS, fname)
    pcm_others_new.convert_df_to_ply(df_pcm_others_new, output_path=output_others_path)

    torch.cuda.empty_cache()

    # Reload phase3 files
    pcm_all = PointCloudModel(max_sh_degree=3)
    pcm_others = PointCloudModel(max_sh_degree=3)
    pcm_all.load_ply(PLY_PATH_ALL, use_train_test_exp=False)
    pcm_others.load_ply(output_others_path, use_train_test_exp=False)

    df_pcm_all = pcm_all.convert_ply_to_df(beta=2.0, alpha=250, id=-1)
    df_pcm_others = pcm_others.convert_ply_to_df(beta=2.0, alpha=250, id=-1)

    # Remove overlapping rows
    ref_cols = ['pos_x', 'pos_y', 'pos_z']
    df_pcm_fog = df_pcm_all[~df_pcm_all[ref_cols].apply(tuple, axis=1)
                            .isin(df_pcm_others[ref_cols].apply(tuple, axis=1))]

    # Save "fog" version
    pcm_fog = PointCloudModel(max_sh_degree=3)
    output_fog_path = os.path.join(PHASE3_FOG, fname)
    pcm_fog.convert_df_to_ply(df_pcm_fog, output_path=output_fog_path)

    torch.cuda.empty_cache()

    print(f"✅ Saved: {output_others_path}, {output_fog_path}")



Processing truck_beta0_alpha000.ply...


100%|██████████| 426/426 [00:23<00:00, 18.17it/s]


871625 / 873403
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/other_gaussian/truck_beta0_alpha000.ply
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta0_alpha000.ply
✅ Saved: /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/other_gaussian/truck_beta0_alpha000.ply, /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta0_alpha000.ply

Processing truck_beta1_alpha025.ply...


100%|██████████| 426/426 [00:13<00:00, 31.80it/s]


516995 / 522798
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/other_gaussian/truck_beta1_alpha025.ply
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta1_alpha025.ply
✅ Saved: /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/other_gaussian/truck_beta1_alpha025.ply, /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta1_alpha025.ply

Processing truck_beta1_alpha050.ply...


...

108424 / 109954
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/other_gaussian/truck_beta6_alpha200.ply
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta6_alpha200.ply
✅ Saved: /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/other_gaussian/truck_beta6_alpha200.ply, /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta6_alpha200.ply

Processing truck_beta6_alpha225.ply...


100%|██████████| 426/426 [00:02<00:00, 167.41it/s]


104779 / 106422
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/other_gaussian/truck_beta6_alpha225.ply
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta6_alpha225.ply
✅ Saved: /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/other_gaussian/truck_beta6_alpha225.ply, /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta6_alpha225.ply

Processing truck_beta6_alpha250.ply...


100%|██████████| 426/426 [00:02<00:00, 167.38it/s]


104170 / 106475
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/other_gaussian/truck_beta6_alpha250.ply
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta6_alpha250.ply
✅ Saved: /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/other_gaussian/truck_beta6_alpha250.ply, /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/truck/fog_gaussian/truck_beta6_alpha250.ply


#### a. train

1. SMALL Scene (Fog Range)

- [1]  y in [-4.70, -1.20]
- [2]  y in [-1.20, -1.10], z in [-2.00,  0.09]+[0.67, 4.00]
- [3]  y in [-1.10, -1.00], z in [-2.00, -0.12]+[0.70, 4.00]
- [4]  y in [-1.00, -0.90], z in [-2.00, -0.28]+[0.86, 4.00]
- [5]  y in [-0.90, -0.80], z in [-2.00, -0.40]+[0.98, 4.00]
- [6]  y in [-0.80, -0.78], z in [-2.00, -0.42]+[1.00, 4.00]
- [7]  y in [-0.78, -0.32], z in [-2.00, -0.35]+[0.90, 4.00]
- [8]  y in [-0.32,  0.00], z in [-2.00, -0.37]+[0.90, 4.00]
- [9]  y in [ 0.00,  0.32], z in [-2.00, -0.37]+[0.88, 4.00]
- [10] y in [ 0.32,  0.53], z in [-2.00, -0.33]+[0.84, 4.00]
- [11] y in [ 0.53,  0.60], z in [ 1.30,  2.55]

- [12] y in [-1.00,  0.55], x in [-4.00, -3.27]+[2.47, 4.00]
- [13] y in [-1.00, -0.30], z in [-0.29, -0.10]+[0.70, 0.83], x in [-3.30, -2.30]+[-1.30, 2.50]

- [-1] y in [ 0.43,  0.60], z in [-5.50,  4.50], x in [-0.85, -0.75]
- [-2] y in [-0.04, -2.70], z in [ 1.32,  1.43], x in [-2.72, -2.90]

- [-3] y in [-0.38,  0.00], x in [-3.57, -3.44], z in [-0.32,  0.82]
- [-4] y in [ 0.00,  0.60], x in [-3.60, -3.20], z in [-0.32,  0.82]
- [-5] y in [ 0.12,  0.60], x in [-3.79, -3.60], z in [ 0.13,  0.37]

- [-6] y in [-0.32,  0.05], x in [ 2.62,  2.76], z in [-0.32,  0.84]
- [-7] y in [ 0.05,  0.60], x in [ 2.47,  2.80], z in [-0.32,  0.84]
- [-8] y in [ 0.15,  0.60], x in [ 2.80,  3.00], z in [ 0.20,  0.40]

- [-9] y in [-1.05, -0.72], x in [-0.40,  1.20], z in [-0.77,  0.15]

In [31]:
PLY_PATHs = glob("/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/train/train*.ply")

# Load the point cloud
pcms = {}
for i, PLY_PATH in enumerate(PLY_PATHs):
    print(f"Loading {PLY_PATH}...")
    key = PLY_PATH.split('/train/train_')[-1].split('.')[0]
    pcm = PointCloudModel(max_sh_degree=3)
    pcm.load_ply(PLY_PATH, use_train_test_exp=False)
    pcms[key] = pcm

pcms = dict(
    sorted(
        pcms.items(),
        key=lambda kv: (
            int(kv[0].split('_')[0].replace('beta', '')),
            int(kv[0].split('_')[1].replace('alpha', ''))
        )
    )
)


len(pcms)

Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/train/train_beta1_alpha250.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/train/train_beta5_alpha100.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/train/train_beta3_alpha125.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/train/train_beta2_alpha150.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/train/train_beta3_alpha225.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/train/train_beta3_alpha250.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/train/train_beta3_alpha175.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/train/train_beta1_alpha025.ply...
Loading /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase2/train/tr

61

In [36]:
target_dir = "/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3"

for fog_params, pcm in pcms.items():
    beta_str, alpha_str = fog_params.split('_')
    beta = int(beta_str.replace("beta", ""))
    alpha = int(alpha_str.replace("alpha", ""))

    df_pcm = pcm.convert_ply_to_df(beta=beta, alpha=alpha, id=-1)

    # ---- Fog ranges (1–13) ----
    fog_cond = (
        # [1]
        ((df_pcm['pos_y'] >= -4.70) & (df_pcm['pos_y'] <= -1.20))
        |
        # [2]
        ((df_pcm['pos_y'] >= -1.20) & (df_pcm['pos_y'] <= -1.10) &
         ((df_pcm['pos_z'].between(-2.00, 0.09)) | (df_pcm['pos_z'].between(0.67, 4.00))))
        |
        # [3]
        ((df_pcm['pos_y'] >= -1.10) & (df_pcm['pos_y'] <= -1.00) &
         ((df_pcm['pos_z'].between(-2.00, -0.12)) | (df_pcm['pos_z'].between(0.70, 4.00))))
        |
        # [4]
        ((df_pcm['pos_y'] >= -1.00) & (df_pcm['pos_y'] <= -0.90) &
         ((df_pcm['pos_z'].between(-2.00, -0.28)) | (df_pcm['pos_z'].between(0.86, 4.00))))
        |
        # [5]
        ((df_pcm['pos_y'] >= -0.90) & (df_pcm['pos_y'] <= -0.80) &
         ((df_pcm['pos_z'].between(-2.00, -0.42)) | (df_pcm['pos_z'].between(1.40, 4.00))))
        |
        # [6]
        ((df_pcm['pos_y'] >= -0.80) & (df_pcm['pos_y'] <= -0.76) &
         ((df_pcm['pos_z'].between(-2.00, -0.44)) | (df_pcm['pos_z'].between(1.02, 4.00))))
        |
        # [7]
        ((df_pcm['pos_y'] >= -0.76) & (df_pcm['pos_y'] <= -0.32) &
         ((df_pcm['pos_z'].between(-2.00, -0.38)) | (df_pcm['pos_z'].between(0.95, 4.00))))
        |
        # [8]
        ((df_pcm['pos_y'] >= -0.32) & (df_pcm['pos_y'] <= 0.00) &
         ((df_pcm['pos_z'].between(-2.00, -0.40)) | (df_pcm['pos_z'].between(0.95, 4.00))))
        |
        # [9]
        ((df_pcm['pos_y'] >= 0.00) & (df_pcm['pos_y'] <= 0.32) &
         ((df_pcm['pos_z'].between(-2.00, -0.40)) | (df_pcm['pos_z'].between(0.90, 4.00))))
        |
        # [10]
        ((df_pcm['pos_y'] >= 0.32) & (df_pcm['pos_y'] <= 0.53) &
         ((df_pcm['pos_z'].between(-2.00, -0.35)) | (df_pcm['pos_z'].between(0.86, 4.00))))
        |
        # [11]
        ((df_pcm['pos_y'] >= 0.53) & (df_pcm['pos_y'] <= 0.60) &
         (df_pcm['pos_z'].between(1.30, 2.55)))
        |
        # [12]
        ((df_pcm['pos_y'] >= -1.20) & (df_pcm['pos_y'] <= 0.55) &
         ((df_pcm['pos_x'].between(-4.00, -3.29)) | (df_pcm['pos_x'].between(2.48, 4.00))))
        |
        # [13]
        ((df_pcm['pos_y'] >= -1.20) & (df_pcm['pos_y'] <= -0.30) &
         ((df_pcm['pos_z'].between(-0.28, -0.12)) | (df_pcm['pos_z'].between(0.70, 0.83))) &
         ((df_pcm['pos_x'].between(-3.30, -2.40)) | (df_pcm['pos_x'].between(-1.30, 2.50))))
    )

    # ---- Non-fog ranges (-1 to -8) ----
    non_fog_cond = (
        # [-1]
        ((df_pcm['pos_y'] >= 0.43) & (df_pcm['pos_y'] <= 0.60) &
         (df_pcm['pos_z'].between(-5.50, 4.50)) &
         (df_pcm['pos_x'].between(-0.85, -0.75)))
        |
        # [-2]
        ((df_pcm['pos_y'] >= -0.04) & (df_pcm['pos_y'] <= 1.00) &
         (df_pcm['pos_z'].between(1.30, 1.43)) &
         (df_pcm['pos_x'].between(-2.90, -2.70)))
        |
        # [-3]
        ((df_pcm['pos_y'] >= -0.38) & (df_pcm['pos_y'] <= 0.00) &
         (df_pcm['pos_x'].between(-3.59, -3.42)) &
         (df_pcm['pos_z'].between(-0.32, 0.82)))
        |
        # [-4]
        ((df_pcm['pos_y'] >= 0.00) & (df_pcm['pos_y'] <= 0.60) &
         (df_pcm['pos_x'].between(-3.60, -3.20)) &
         (df_pcm['pos_z'].between(-0.34, 0.82)))
        |
        # [-5]
        ((df_pcm['pos_y'] >= 0.12) & (df_pcm['pos_y'] <= 0.60) &
         (df_pcm['pos_x'].between(-3.80, -3.60)) &
         (df_pcm['pos_z'].between(0.11, 0.42)))
        |
        # [-6]
        ((df_pcm['pos_y'] >= -0.32) & (df_pcm['pos_y'] <= 0.05) &
         (df_pcm['pos_x'].between(2.62, 2.76)) &
         (df_pcm['pos_z'].between(-0.32, 0.84)))
        |
        # [-7]
        ((df_pcm['pos_y'] >= 0.05) & (df_pcm['pos_y'] <= 0.60) &
         (df_pcm['pos_x'].between(2.47, 2.81)) &
         (df_pcm['pos_z'].between(-0.32, 0.84)))
        |
        # [-8]
        ((df_pcm['pos_y'] >= 0.15) & (df_pcm['pos_y'] <= 0.60) &
         (df_pcm['pos_x'].between(2.80, 3.00)) &
         (df_pcm['pos_z'].between(0.05, 0.40)))
        |
        # [-9]
        ((df_pcm['pos_y'] >= -1.05) & (df_pcm['pos_y'] <= -0.72) &
         (df_pcm['pos_x'].between(-0.40, 1.20)) &
         (df_pcm['pos_z'].between(-0.34, 1.00)))
        # [-10]
        |
        ((df_pcm['pos_y'] >= 0.1) & (df_pcm['pos_y'] <= 0.2) &
         (df_pcm['pos_x'].between(2.90, 3.00)) &
         (df_pcm['pos_z'].between(-0.15,  0.60)))
    )

    # Final condition: fog but not in non-fog
    final_cond = fog_cond & (~non_fog_cond)

    df_pcm_fog    = df_pcm[final_cond]
    df_pcm_others = df_pcm[~final_cond]

    pcm.convert_df_to_ply(df_pcm_fog, output_path=f"{target_dir}/fog_gaussian/train_{fog_params}.ply")
    pcm.convert_df_to_ply(df_pcm_others, output_path=f"{target_dir}/other_gaussian/train_{fog_params}.ply")


Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/fog_gaussian/train_beta0_alpha000.ply
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/other_gaussian/train_beta0_alpha000.ply
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/fog_gaussian/train_beta1_alpha025.ply
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/other_gaussian/train_beta1_alpha025.ply
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/fog_gaussian/train_beta1_alpha050.ply
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/other_gaussian/train_beta1_alpha050.ply
Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phase3/fog_gaussian/train_beta1_alpha075.ply
Saved PLY format Splat to /home/tsy/Docume

### TEST

In [ ]:
df_beta3_alpha100 = pcms['beta3_alpha100'].convert_ply_to_df(beta=3, alpha=100, id=1)

df_beta3_alpha100.head()


,pos_x,pos_y,pos_z,f_dc_0,f_dc_1,f_dc_2,scale_x,scale_y,scale_z,rot_0,rot_1,rot_2,rot_3,opacity,beta,alpha,is_fog,id
0,-46.056801,-34.672028,-63.476521,2.836292,2.631686,3.397916,-3.671977,-1.081934,-1.989728,0.748493,0.480895,0.396542,-0.184054,11.522381,3.0,100.0,1.0,1.0
1,-44.530788,-32.114323,-60.636570,2.179316,1.818048,1.996933,-0.584879,-2.397278,-0.334345,0.775586,0.089988,0.260339,0.402535,1.167846,3.0,100.0,1.0,1.0
2,-26.614662,-64.278282,-102.678543,2.001575,2.431832,2.868317,-1.392875,-1.441928,-2.591662,0.810607,0.431341,0.377246,-0.136567,11.596570,3.0,100.0,1.0,1.0
3,-6.919754,-57.027378,-98.698181,0.160735,0.514788,0.439602,-2.509508,-2.164816,-1.197374,1.086890,-0.381903,0.281751,0.081070,1.228123,3.0,100.0,1.0,1.0
4,-31.011082,-50.708141,-88.337219,5.292072,5.673431,5.086677,-1.078711,-2.199310,-1.050936,0.950921,-0.261886,0.589441,-0.078695,14.635775,3.0,100.0,1.0,1.0


In [ ]:
pos_x_range = [df_beta3_alpha100['pos_x'].min(), df_beta3_alpha100['pos_x'].max()]
pos_y_range = [df_beta3_alpha100['pos_y'].min(), df_beta3_alpha100['pos_y'].max()]
pos_z_range = [df_beta3_alpha100['pos_z'].min(), df_beta3_alpha100['pos_z'].max()]

pos_x_range, pos_y_range, pos_z_range

([np.float32(-64.39654), np.float32(39.54907)],
 [np.float32(-64.27828), np.float32(2.5767338)],
 [np.float32(-104.72168), np.float32(23.47043)])

In [ ]:
# x_range: -9.0, 9.0
# y_range: -6.0, 1.2
# z_range: -4.5, 7.0

condition_x = (df_beta3_alpha100['pos_x'] >= -9.0) * (df_beta3_alpha100['pos_x'] <= 9.0)
condition_y = (df_beta3_alpha100['pos_y'] >= -6.0) * (df_beta3_alpha100['pos_y'] <= 1.2)
condition_z = (df_beta3_alpha100['pos_z'] >= -4.5) * (df_beta3_alpha100['pos_z'] <= 7.0)

condition_all = condition_x * condition_y * condition_z

df_filtered_beta3_alpha100 = df_beta3_alpha100[condition_x * condition_y * condition_z]

df_filtered_beta3_alpha100

,pos_x,pos_y,pos_z,f_dc_0,f_dc_1,f_dc_2,scale_x,scale_y,scale_z,rot_0,rot_1,rot_2,rot_3,opacity,beta,alpha,is_fog,id
304,-8.706302,-0.043249,-2.805046,5.170978,5.422734,1.741534,-2.476408,-5.653971,-1.630299,1.297176,0.036881,0.523487,0.256200,2.161807,3.0,100.0,1.0,1.0
307,-8.815155,0.070249,-2.890392,4.758027,4.747109,2.459497,-3.273002,-2.648410,-2.174233,0.553378,-0.214727,0.348727,0.464513,9.074882,3.0,100.0,1.0,1.0
308,-8.910439,0.130163,-3.107243,3.682331,3.652752,2.806051,-8.506537,-2.255462,-0.472634,0.845871,-0.298374,0.340120,-0.516720,11.475802,3.0,100.0,1.0,1.0
493,-4.697136,-1.973029,3.515457,0.918910,0.468853,0.399992,-2.346923,-0.514094,-1.137186,0.598361,-0.377375,-0.354777,0.246527,-0.914073,3.0,100.0,1.0,1.0
494,-5.007823,-1.767274,3.595255,0.517293,0.580853,1.452052,-3.183904,-0.197682,-1.667788,0.386239,-0.360675,0.549064,-0.330307,-3.405715,3.0,100.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299741,8.212709,1.061577,1.951370,1.925145,2.019717,2.314161,-2.684040,-0.524382,-3.607005,0.714342,0.238126,-0.118436,0.553814,4.653973,3.0,100.0,1.0,1.0
299745,8.980007,1.110841,1.278501,-0.545040,-0.335598,0.416770,-3.145357,-1.437862,-3.027044,0.820305,0.246117,-0.080175,0.232776,-0.329632,3.0,100.0,1.0,1.0
300379,7.993296,-0.101576,6.954078,-0.031856,-0.028783,-0.487738,-2.087306,-0.863478,-2.780468,0.719635,-0.273362,0.029018,0.631205,9.070639,3.0,100.0,1.0,1.0
300396,8.095436,0.048370,6.871426,0.481622,0.374533,0.217248,-3.223332,-0.863309,-2.887905,0.605112,-0.399958,-0.177092,-0.551114,-0.414254,3.0,100.0,1.0,1.0


In [ ]:
target_dir = "/home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phrase2/train"
pcm.convert_df_to_ply(df_filtered_beta3_alpha100, output_path=f"{target_dir}/train_f_beta3_alpha100.ply")

Saved PLY format Splat to /home/tsy/Documents/TeamM_Defog/kpro-dehaze/data/train/processed/phrase2/train/train_f_beta3_alpha100.ply
